In [4]:
import os
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D

In [5]:
Train_Dir = 'images/train'
Test_Dir = 'images/test'

### Function is used to collect file paths and associated labels from a directory structure where each subdirectory corresponds to a different class, and images within each subdirectory belong to that class.

In [7]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels
 

In [10]:
train = pd.DataFrame()
train['image'],train['label'] = createdataframe(Train_Dir)
print(train)

angry completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                              image     label
0          images/train\angry\0.jpg     angry
1          images/train\angry\1.jpg     angry
2         images/train\angry\10.jpg     angry
3       images/train\angry\1015.jpg     angry
4       images/train\angry\1016.jpg     angry
...                             ...       ...
2231  images/train\surprise\894.jpg  surprise
2232  images/train\surprise\909.jpg  surprise
2233  images/train\surprise\960.jpg  surprise
2234  images/train\surprise\964.jpg  surprise
2235  images/train\surprise\972.jpg  surprise

[2236 rows x 2 columns]


In [12]:
test = pd.DataFrame()
test['image'],test['label'] = createdataframe(Test_Dir)
print(test)

angry completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                             image     label
0       images/test\angry\1024.jpg     angry
1       images/test\angry\1071.jpg     angry
2       images/test\angry\1106.jpg     angry
3       images/test\angry\1124.jpg     angry
4       images/test\angry\1127.jpg     angry
..                             ...       ...
983  images/test\surprise\9806.jpg  surprise
984  images/test\surprise\9830.jpg  surprise
985  images/test\surprise\9853.jpg  surprise
986  images/test\surprise\9878.jpg  surprise
987   images/test\surprise\993.jpg  surprise

[988 rows x 2 columns]


## Data Preprocessing

In [15]:
"""
The tqdm library is used to create progress bars, 
allowing you to visualize the progress.

"""
from tqdm.notebook import tqdm

In [21]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features
        

## Feature Extraction

In [22]:
"""
The function processes a list of image file paths, loads and converts them to grayscale,
and organizes them into a NumPy array suitable for further processing.

"""
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

  0%|          | 0/2236 [00:00<?, ?it/s]

C:\Users\Star\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


  0%|          | 0/988 [00:00<?, ?it/s]

## Normalization
#### Normalize the pixel values of the image features. This normalization is a common preprocessing step in machine learning, especially when working with neural networks.

In [23]:
x_train = train_features / 255.0
x_test = test_features / 255.0

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)


## Building the Neural Network Model

In [28]:
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# flattening
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

# model compilation
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )


## Training the Model

In [ ]:
model.fit(x=x_train, y =y_train, batch_size=128, epochs=100, validation_data=(x_train,y_test))

## Save the Model